# 📊 Análisis EDA - training_dataset.csv

## Prerequisito: Valido esquema antes de analizar

In [ ]:
import pandas as pd
import os
from validators.validation_schema import validate_training_dataset
import dotenv
dotenv.load_dotenv()

TRANING_DATA_PATH = os.getenv("TRAINING_DATA_PATH")

if not TRANING_DATA_PATH:
    raise ValueError("La variable de entorno 'TRAINING_DATA_PATH' no está definida.")

if not os.path.exists(TRANING_DATA_PATH):
    raise FileNotFoundError(f"La ruta especificada '{TRANING_DATA_PATH}' no existe.")

# Cargar dataset
df = pd.read_csv(TRANING_DATA_PATH)

# Validar estructura y contenido
df = validate_training_dataset(df)
print("✅ Dataset validado correctamente con pandera.")


## Seleccionar partidas de un determinado jugador

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar dataset
df = pd.read_csv('/app/src/data/training_dataset.csv')
#df.head()

df.describe()


,move_number,material_balance,material_total,num_pieces,branching_factor,self_mobility,opponent_mobility,player_color,has_castling_rights,move_number_global,is_repetition,is_low_mobility,tags,score_diff
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.0,94.00000,0.0,0.0
mean,13.978723,-0.888298,71.771277,12.340426,63.968085,31.819149,32.148936,0.510638,0.585106,27.446809,0.0,0.06383,NaN,NaN
std,9.190901,1.806357,7.886089,1.920759,13.131635,10.240089,10.103717,0.502567,0.495346,18.376947,0.0,0.24576,NaN,NaN
min,1.000000,-9.250000,52.250000,8.000000,27.000000,3.000000,3.000000,0.000000,0.000000,1.000000,0.0,0.00000,NaN,NaN
25%,6.250000,-1.250000,66.500000,11.000000,59.000000,29.000000,30.000000,0.000000,0.000000,12.250000,0.0,0.00000,NaN,NaN
50%,12.000000,-0.500000,73.750000,13.000000,67.000000,33.000000,33.500000,1.000000,1.000000,24.000000,0.0,0.00000,NaN,NaN
75%,21.000000,0.000000,79.000000,14.000000,71.750000,37.000000,37.000000,1.000000,1.000000,41.750000,0.0,0.00000,NaN,NaN
max,33.000000,2.250000,79.000000,14.000000,95.000000,51.000000,51.000000,1.000000,1.000000,65.000000,0.0,1.00000,NaN,NaN


## 1. Distribución de etiquetas de error

In [ ]:

df['error_label'].value_counts().plot(kind='bar', color='skyblue')
plt.title("Distribución de etiquetas de error")
plt.xlabel("Etiqueta")
plt.ylabel("Cantidad")
plt.grid(True)
plt.show()


## 2. Correlaciones entre variables estratégicas y errores

In [ ]:

variables = ['branching_factor', 'self_mobility', 'opponent_mobility', 'material_total']
correlation_matrix = df[variables + ['error_label']].corr(numeric_only=True)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Matriz de correlación")
plt.show()


## 3. Análisis por control del centro vs errores

In [ ]:

pd.crosstab(df['is_center_controlled'], df['error_label']).plot(kind='bar', stacked=True)
plt.title("Errores según control del centro")
plt.xlabel("¿Controla el centro?")
plt.ylabel("Cantidad")
plt.grid(True)
plt.show()


## 4. Fase de juego vs tipo de error

In [ ]:

pd.crosstab(df['phase'], df['error_label']).plot(kind='bar', stacked=True)
plt.title("Errores por fase de juego")
plt.xlabel("Fase")
plt.ylabel("Cantidad")
plt.grid(True)
plt.show()


## 5. Boxplot de movilidad vs error

In [ ]:

sns.boxplot(data=df, x='error_label', y='self_mobility')
plt.title("Movilidad propia según tipo de error")
plt.grid(True)
plt.show()


## 6. Error promedio en baja movilidad

In [ ]:
sns.barplot(data=df, x="is_low_mobility", y="score_diff", hue="classification")
plt.title("score_diff en posiciones con baja movilidad")
plt.tight_layout()
plt.show()

## 7. Correlaciones numéricas

In [ ]:
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Mapa de calor de correlaciones")
plt.tight_layout()
plt.show()

## 8. Conclusiones preliminares

- Las jugadas con baja movilidad muestran mayor score_diff promedio (peores resultados).
- Existen correlaciones entre movilidad propia, balance de material y score_diff.
- La distribución de errores es más frecuente en fases medias y finales con menos opciones legales.